In [1]:
import os
import torch
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.core.display import display, HTML
from difflib import ndiff
import time
import gc
from tqdm import tqdm

# Hugging Face и Unsloth
from datasets import load_dataset, Dataset
from transformers import TextStreamer, BitsAndBytesConfig
from unsloth import FastLanguageModel
from trl import SFTTrainer, SFTConfig
from peft import LoraConfig, PeftModel, get_peft_model
from transformers import AutoTokenizer

# Weights & Biases
import wandb

# Мои функции
from utils import *

/tmp/ipykernel_1274/1751918087.py:6: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython.display
  from IPython.core.display import display, HTML
/opt/conda/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/tmp/ipykernel_1274/1751918087.py:15: UserWarning: WARNING: Unsloth should be imported before transformers to ensure all optimizations are applied. Your code may run slower or encounter memory issues without these optimizations.

Please restructure your imports with 'import unsloth' at the top of your file.
  from unsloth import FastLanguageModel


🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


2025-05-19 01:08:43.566751: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-05-19 01:08:43.566885: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-05-19 01:08:43.674916: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-05-19 01:08:43.909459: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-05-19 01:08:45.744370: W tensorflow/compiler/tf2

🦥 Unsloth Zoo will now patch everything to make training faster!


In [2]:
# Настройки
HF_ACCESS_TOKEN = ""
WANDB_PROJECT_NAME = "quantization_experiments"
WANDB_NOTEBOOK_NAME = 'text-correction-experiments' # Имя этого ноутбука для WandB

# Логин в WandB
try:
    WANDB_API_KEY = ''
    wandb.login(key=WANDB_API_KEY)
    anonymous = None
except:
    anonymous = "must" # или "allow" или "never"
    wandb.login(anonymous=anonymous, relogin=True)

print(f"Torch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"CUDA version: {torch.version.cuda}")
    print(f"Current device: {torch.cuda.current_device()}")
    print(f"Device name: {torch.cuda.get_device_name(torch.cuda.current_device())}")

MAX_SEQ_LENGTH = 2048 # Длина контекста
TARGET_MODULES = ["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"] # Модули для LoRA

# Имена моделей
# Для Эксперимента 1: Полная модель. Замените на точное имя 8B модели, если оно отличается.
BASE_MODEL_NAME_FULL_PRECISION = "unsloth/Qwen3-0.6B"
# Для Эксперимента 3: Предварительно квантованная модель от Unsloth
BASE_MODEL_NAME_UNSLOTH_QUANTIZED = "unsloth/Qwen3-0.6B-unsloth-bnb-4bit"

# Директории для сохранения моделей
OUTPUT_DIR_LORA_FULL = "./results_lora_full_precision"
OUTPUT_DIR_PTQ_8BIT = "./results_ptq_8bit"
OUTPUT_DIR_PTQ_4BIT = "./results_ptq_4bit"
OUTPUT_DIR_QLORA = "./results_qlora"
MERGED_MODEL_DIR_FOR_PTQ = "./merged_lora_full_model_for_ptq"

# Список для сбора результатов
all_experiment_results = []

wandb: ERROR Failed to detect the name of this notebook. You can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: galkin-vova1 (galkin-vova42) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Torch version: 2.7.0+cu126
CUDA available: True
CUDA version: 12.6
Current device: 0
Device name: NVIDIA GeForce RTX 4090


In [3]:
# Загрузка датасета
non_reasoning_dataset = load_dataset("galkinv42/text-correction-ru", token=HF_ACCESS_TOKEN)
dataset_split = non_reasoning_dataset['train'].train_test_split(test_size=0.2, seed=42)

dataset_train_raw = dataset_split["train"]
dataset_test_raw = dataset_split["test"]

# Системный промпт и форматирование
SYSTEM_PROMPT = 'Отредактируйте следующий текст. Очень важно, чтобы вы сохранили как можно больше оригинального текста. Вы можете вносить небольшие и нечастые правки, которые исправляют только грамматические, орфографические или пунктационные ошибки. Если в предложении нет ошибок, повторите предложение дословно. Не объясняйте свои правки и не добавляйте примечания. Используйте только русский язык.'

def format_chatml(example):
    return {
        "conv": [
            {"role": "system", "content": SYSTEM_PROMPT},
            {"role": "user", "content": example["text_with_errors"]},
            {"role": "assistant", "content": example["corrected_text"]}
        ]
    }

dataset_train_formatted = dataset_train_raw.map(format_chatml)
dataset_test_formatted = dataset_test_raw.map(format_chatml)

# Функция форматирования будет применена позже, непосредственно перед обучением,
# так как токенизатор может меняться для разных моделей.
def formatting_prompts_func(examples, tokenizer_to_use):
    convos = examples["conv"]
    texts = [tokenizer_to_use.apply_chat_template(convo, tokenize=False, add_generation_prompt=False) for convo in convos]
    return {"text": texts}

print(f"Размер обучающего набора: {len(dataset_train_raw)}")
print(f"Размер тестового набора: {len(dataset_test_raw)}")
print("\nПример из обучающего набора (сырой):")
print(dataset_train_raw[0])
print("\nПример из обучающего набора (форматированный ChatML):")
print(dataset_train_formatted[0]["conv"])

Размер обучающего набора: 555
Размер тестового набора: 139

Пример из обучающего набора (сырой):
{'text_with_errors': 'Глобальное потипление является одной из наиболее актуальных проблем современности. Средняя температура на планете неуклонно повышается что приводит к таянию ледников повышению уровня мирового океана и изменению климата. Основной причиной этого явления считается выброс парниковых газов в атмосферу в результате человеческой деятельности. Сжигание ископаемого топлива вырубка лесов и промышленное производство значительно увеличивают концентрацию углекислого газа в воздухе. Последствия глобального потепления могут быть катастрофическими: затопление прибрежных территорий усиление засух и наводнений исчезновение многих видов животных и растений. Для решения этой проблемы необходимы совместные усилия всех стран направленные на сокращение выбросов парниковых газов развитие альтернативных источников энергии и сохранение лесов. Каждый человек также может внести свой вклад в борьб

### Эксперимент 1: Обучение LoRa на "полной" модели Qwen (без предварительной квантизации)

Мы будем использовать Qwen/Qwen2-7B-Instruct в качестве примера. Unsloth позволяет загружать и неквантованные модели и применять к ним LoRA.

In [4]:
def run_experiment_1_full_lora():
    print("\n--- Запуск Эксперимента 1: LoRa на полной модели ---")
    
    run_name_wandb = "exp1_lora_full_precision"
    wandb.init(entity='galkin-vova42', project=WANDB_PROJECT_NAME, name=run_name_wandb, job_type="train", config={"experiment_type": "lora_full_precision", "base_model": BASE_MODEL_NAME_FULL_PRECISION}, reinit=False)

    # Загрузка модели и токенизатора БЕЗ квантизации
    # Unsloth также оптимизирует загрузку полных моделей
    model, tokenizer = FastLanguageModel.from_pretrained(
        model_name=BASE_MODEL_NAME_FULL_PRECISION,
        max_seq_length=MAX_SEQ_LENGTH,
        dtype=None, # Автоматический выбор или torch.float16, torch.bfloat16
        load_in_4bit=False, # ВАЖНО: без 4-битной квантизации
        load_in_8bit=False, # ВАЖНО: без 8-битной квантизации
        token=HF_ACCESS_TOKEN,
    )
    
    # Подготовка датасета с текущим токенизатором
    dataset_train = Dataset.from_dict(formatting_prompts_func(dataset_train_formatted, tokenizer))
    dataset_test = Dataset.from_dict(formatting_prompts_func(dataset_test_formatted, tokenizer))

    # Добавление LoRA адаптеров
    model = FastLanguageModel.get_peft_model(
        model,
        r=32,
        target_modules=TARGET_MODULES,
        lora_alpha=32, # Обычно alpha = r или 2*r
        lora_dropout=0,
        bias="none",
        use_gradient_checkpointing="unsloth",
        random_state=42,
        use_rslora=False,
        loftq_config=None,
    )
    
    trainer_config = SFTConfig(
        dataset_text_field="text",
        output_dir=OUTPUT_DIR_LORA_FULL,
        per_device_train_batch_size=3,
        per_device_eval_batch_size=3,
        gradient_accumulation_steps=4, # Увеличиваем GA
        warmup_steps=10, # Увеличено для стабильности
        do_eval=True,
        eval_steps=20, # Чаще оценка
        save_steps=20,
        save_total_limit=2, # Сохраняем лучшую и последнюю
        greater_is_better=False,
        load_best_model_at_end=True,
        eval_strategy='steps',
        metric_for_best_model="eval_loss",
        max_steps=100,
        learning_rate=1e-4, # Может потребоваться подстройка (1e-4 to 2e-5)
        logging_steps=1,
        optim="adamw_torch", # Оптимизатор для неквантованных моделей
        weight_decay=0.01,
        lr_scheduler_type="linear",
        seed=42,
        report_to="wandb",
        fp16=not torch.cuda.is_bf16_supported(), # Используем fp16 если bf16 не доступен
        bf16=torch.cuda.is_bf16_supported(),
    )

    trainer = SFTTrainer(
        model=model,
        tokenizer=tokenizer,
        train_dataset=dataset_train,
        eval_dataset=dataset_test,
        args=trainer_config,
    )

    # Память до обучения
    gpu_stats = torch.cuda.get_device_properties(0) if torch.cuda.is_available() else None
    start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024**3, 3) if torch.cuda.is_available() else 0
    
    print("Начало обучения LoRa на полной модели...")
    trainer_stats = trainer.train()
    
    # Память после обучения
    if torch.cuda.is_available():
        used_memory = round(torch.cuda.max_memory_reserved() / 1024**3, 3)
        used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
        print(f"Пиковая VRAM для обучения (полная LoRa): {used_memory_for_lora:.3f} GB")
        wandb.log({"peak_vram_train_gb": used_memory_for_lora})


    # Сохранение лучшей модели (адаптеров)
    # SFTTrainer с load_best_model_at_end=True уже загрузил лучшую модель
    best_model_path = os.path.join(OUTPUT_DIR_LORA_FULL, "best_lora_adapters")
    model.save_pretrained(best_model_path)
    tokenizer.save_pretrained(best_model_path)
    print(f"Лучшие LoRa адаптеры сохранены в: {best_model_path}")
    wandb.save(os.path.join(best_model_path, "*")) # Сохраняем артефакты в wandb

    print("Оценка лучшей модели (пакетная обработка)...")
    
    predictions = []
    references_batch_eval = [] # Переименовано, чтобы избежать конфликта с глобальной references
    sample_inference_latencies = [] # Будет хранить время на обработку одного батча

    # Используем небольшой срез тестового датасета для быстрой оценки
    num_samples_for_eval = min(100, len(dataset_test_raw)) # Увеличьте для более точной оценки
    eval_batch_size = 8

    # Подготовка данных для оценки
    eval_data = []
    for i in range(num_samples_for_eval):
        example = dataset_test_raw[i]
        eval_data.append({
            "user_content": example["text_with_errors"],
            "reference_text": example["corrected_text"]
        })

    # Память перед инференсом (модель уже загружена)
    if torch.cuda.is_available():
        torch.cuda.reset_peak_memory_stats() # Сброс статистики для замера пиковой памяти на инференс
        initial_inference_vram = torch.cuda.max_memory_allocated()
    else:
        initial_inference_vram = 0

    device = "cuda" if torch.cuda.is_available() else "cpu"

    for batch_data in tqdm(create_batches(eval_data, eval_batch_size)):
        batch_prompts_text = []
        current_references = []

        for item in batch_data:
            messages = [
                {"role": "system", "content": SYSTEM_PROMPT},
                {"role": "user", "content": item["user_content"]}
            ]
            prompt_text = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
            batch_prompts_text.append(prompt_text)
            current_references.append(item["reference_text"].strip())
        
        inputs = tokenizer(
            batch_prompts_text, 
            return_tensors="pt", 
            padding=True,
            truncation=True,
            max_length=tokenizer.model_max_length if tokenizer.model_max_length < MAX_SEQ_LENGTH else MAX_SEQ_LENGTH # Используем меньшее из двух
        ).to(device)

        start_time = time.time()
        with torch.no_grad():
            outputs = model.generate(
                **inputs, 
                max_new_tokens=min(512, MAX_SEQ_LENGTH),
                temperature=0.01,
                do_sample=False,
                pad_token_id=tokenizer.pad_token_id # Явно указываем pad_token_id
            )
        end_time = time.time()
        sample_inference_latencies.append(end_time - start_time) # Время на весь батч
        
        input_ids_lengths = inputs.attention_mask.sum(dim=1).tolist() # Длины реальных токенов в промпте (без паддинга)

        batch_output_texts = []
        for i in range(outputs.shape[0]):
            
            generated_tokens_only = outputs[i, input_ids_lengths[i]:]
            output_text = tokenizer.decode(generated_tokens_only, skip_special_tokens=True)
            batch_output_texts.append(output_text.strip())
            
        predictions.extend(batch_output_texts)
        references_batch_eval.extend(current_references)

    if torch.cuda.is_available():
        peak_inference_vram_bytes = torch.cuda.max_memory_allocated() 
        peak_inference_vram = peak_inference_vram_bytes / (1024**3)
    else:
        peak_inference_vram = 0
    
    bleu_score = calculate_bleu(predictions, references_batch_eval)
    cer_score = calculate_cer(predictions, references_batch_eval)
    
    total_inference_time = sum(sample_inference_latencies)
    avg_latency_per_sample = total_inference_time / num_samples_for_eval if num_samples_for_eval > 0 else -1


    print(f"BLEU ({model_name_str if 'model_name_str' in locals() else 'Current Model'}): {bleu_score:.4f}")
    print(f"CER ({model_name_str if 'model_name_str' in locals() else 'Current Model'}): {cer_score:.4f}")
    print(f"Avg Inference Latency per sample ({model_name_str if 'model_name_str' in locals() else 'Current Model'}): {avg_latency_per_sample:.4f} s")
    print(f"Peak VRAM during batched inference ({model_name_str if 'model_name_str' in locals() else 'Current Model'}): {peak_inference_vram:.3f} GB")


    trainable_params, total_params = get_model_trainable_parameters(model)
    
    results = {
        "experiment_group": "LoRa Full Precision",
        "model_name": f"LoRa ({BASE_MODEL_NAME_FULL_PRECISION})",
        "val_loss": trainer.state.best_model_checkpoint_loss if hasattr(trainer.state, "best_model_checkpoint_loss") else trainer_stats.training_loss,
        "bleu": bleu_score,
        "cer": cer_score,
        "trainable_params": trainable_params,
        "total_params": total_params,
        "vram_train_peak_gb": used_memory_for_lora if torch.cuda.is_available() else 0,
        "vram_inference_peak_gb": peak_inference_vram,
        "avg_inference_latency_s": avg_latency_per_sample,
        "model_path": best_model_path,
    }
    all_experiment_results.append(results)
    wandb.log({
        "eval_bleu": bleu_score, "eval_cer": cer_score, "avg_inference_latency": avg_latency_per_sample,
        "trainable_params": trainable_params, "total_params": total_params,
        "peak_inference_vram_gb": peak_inference_vram
    })
    
    wandb.finish()
    
    # Очистка
    clear_gpu_memory(model, trainer)
    return best_model_path, tokenizer

In [5]:
path_to_lora_full_adapters, tokenizer_lora_full = run_experiment_1_full_lora()
print(f"Эксперимент 1 завершен. Адаптеры в: {path_to_lora_full_adapters}")

wandb: WARNING Using a boolean value for 'reinit' is deprecated. Use 'return_previous' or 'finish_previous' instead.



--- Запуск Эксперимента 1: LoRa на полной модели ---


==((====))==  Unsloth 2025.5.6: Fast Qwen3 patching. Transformers: 4.51.3.
   \\   /|    NVIDIA GeForce RTX 4090. Num GPUs = 1. Max memory: 23.526 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.7.0+cu126. CUDA: 8.9. CUDA Toolkit: 12.6. Triton: 3.3.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.30. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Unsloth 2025.5.6 patched 28 layers with 28 QKV layers, 28 O layers and 28 MLP layers.
Unsloth: Tokenizing ["text"] (num_proc=2): 100%|██████████| 139/139 [00:01<00:00, 134.40 examples/s]


Начало обучения LoRa на полной модели...


...

Лучшие LoRa адаптеры сохранены в: ./results_lora_full_precision/best_lora_adapters
Оценка лучшей модели (пакетная обработка)...


0it [00:00, ?it/s]A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.
/opt/conda/lib/python3.10/site-packages/unsloth/kernels/utils.py:438: UserWarning: An output with one or more elements was resized since it had shape [1, 8, 1024], which does not match the required output shape [8, 1, 1024]. This behavior is deprecated, and in a future PyTorch release outputs will not be resized unless they have zero elements. You can explicitly reuse an out tensor t by resizing it, inplace, to zero elements with t.resize_(0). (Triggered internally at /pytorch/aten/src/ATen/native/Resize.cpp:30.)
  out = torch_matmul(X, W.t(), out = out)
1it [00:21, 21.02s/it]A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.
2it [00:41, 20.88s/it]A decoder-only architecture is 

BLEU (Current Model): 0.1648
CER (Current Model): 0.8239
Avg Inference Latency per sample (Current Model): 2.6946 s
Peak VRAM during batched inference (Current Model): 3.524 GB


avg_inference_latency,▁
eval/loss,█▃▂▁▁
eval/runtime,█▁▁▂▁
eval/samples_per_second,▁██▇█
eval/steps_per_second,▁██▇█
eval_bleu,▁
eval_cer,▁
peak_inference_vram_gb,▁
peak_vram_train_gb,▁
total_params,▁
train/epoch,▁▁▁▁▁▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇▇██


GPU memory cleared.
Эксперимент 1 завершен. Адаптеры в: ./results_lora_full_precision/best_lora_adapters


### QLora

In [6]:
def run_experiment_3_qlora(results_accumulator_list):
    print("\n--- Запуск Эксперимента 3: QLoRa ---")
    
    ranks = [16, 32] 
    alphas_map = {16: [32], 32: [64]}

    best_params_qlora = {"r": 0, "alpha": 0, "loss": float('inf')}
    
    # Загрузка базовой квантованной модели и токенизатора один раз для HPO
    base_q_model_for_hpo, tokenizer_qlora = FastLanguageModel.from_pretrained(
        model_name=BASE_MODEL_NAME_UNSLOTH_QUANTIZED,
        max_seq_length=MAX_SEQ_LENGTH,
        load_in_4bit=True,
        token=HF_ACCESS_TOKEN,
    )
    
    dataset_train_q = Dataset.from_dict(formatting_prompts_func(dataset_train_formatted, tokenizer_qlora))
    dataset_test_q = Dataset.from_dict(formatting_prompts_func(dataset_test_formatted, tokenizer_qlora))

    print("Подбор гиперпараметров r и alpha для QLoRa...")
    for r_val in ranks:
        for alpha_val in alphas_map[r_val]:
            current_run_name_hpo = f"qlora_hpo_r{r_val}_a{alpha_val}"
            print(f"\nТестирование QLoRa с r={r_val}, alpha={alpha_val}")
            
            # Для каждой HPO итерации - новый run
            wandb.init(entity='galkin-vova42', project=WANDB_PROJECT_NAME, name=current_run_name_hpo, job_type="hpo_qlora", 
                       config={"experiment_type": "qlora_hpo", "r": r_val, "alpha": alpha_val, "base_model": BASE_MODEL_NAME_UNSLOTH_QUANTIZED},
                       reinit=True)
            
            # Создаем PEFT модель. Unsloth должен корректно обрабатывать применение новых адаптеров.
            # Для полной уверенности можно перезагружать base_q_model_for_hpo, но попробуем так.
            # Если будут проблемы с "наслоением" адаптеров, потребуется перезагрузка базовой модели в цикле.
            # Unsloth FastLanguageModel.get_peft_model должен создавать новые адаптеры.
            
            # Чтобы избежать модификации base_q_model_for_hpo, если get_peft_model работает inplace (хотя не должен)
            # или если предыдущие адаптеры не удаляются корректно, можно было бы загружать temp_base_model,
            # но это замедлит HPO. Полагаемся на Unsloth.
            if 'peft_q_model_hpo' in locals(): # Очистка предыдущей HPO модели
                 clear_gpu_memory(peft_q_model_hpo)

            peft_q_model_hpo = FastLanguageModel.get_peft_model(
                base_q_model_for_hpo, # Используем одну и ту же базовую квантованную модель
                r=r_val,
                target_modules=TARGET_MODULES,
                lora_alpha=alpha_val,
                lora_dropout=0,
                bias="none",
                use_gradient_checkpointing="unsloth", # "unsloth" предпочтительнее True
                random_state=42,
            )

            trainer_q_hpo = SFTTrainer(
                model=peft_q_model_hpo,
                tokenizer=tokenizer_qlora,
                train_dataset=dataset_train_q,
                eval_dataset=dataset_test_q, 
                args=SFTConfig(
                    dataset_text_field="text",
                    output_dir=f"./results_qlora_hpo_r{r_val}_a{alpha_val}", # Временная директория
                    per_device_train_batch_size=2, # Как в вашем оригинальном коде для QLoRA
                    gradient_accumulation_steps=4,
                    warmup_steps=5,
                    max_steps=30, 
                    learning_rate=2e-4,
                    logging_steps=5,
                    optim="adamw_8bit", 
                    weight_decay=0.01,
                    lr_scheduler_type="linear",
                    seed=42,
                    report_to="wandb",
                    do_eval=True,
                    eval_strategy="steps",
                    eval_steps=10,
                    save_strategy="no", 
                ),
            )
            
            trainer_q_hpo.train()
            current_eval_loss = float('inf')
            for log_entry in reversed(trainer_q_hpo.state.log_history):
                if 'eval_loss' in log_entry:
                    current_eval_loss = log_entry['eval_loss']
                    break
            
            print(f"HPO QLoRa r={r_val}, alpha={alpha_val} -> eval_loss: {current_eval_loss}")
            wandb.log({"hpo_eval_loss": current_eval_loss, "r": r_val, "alpha": alpha_val})

            if current_eval_loss < best_params_qlora["loss"]:
                best_params_qlora["loss"] = current_eval_loss
                best_params_qlora["r"] = r_val
                best_params_qlora["alpha"] = alpha_val
            
            wandb.finish()
            # Очищаем только peft_q_model_hpo и trainer, базовая модель и токенизатор остаются для следующей итерации HPO
            clear_gpu_memory(peft_q_model_hpo, trainer_q_hpo) 

    # Очищаем базовую модель и токенизатор, использованные для HPO
    clear_gpu_memory(base_q_model_for_hpo) # tokenizer_qlora понадобится для финального обучения

    print(f"\nЛучшие параметры для QLoRa: r={best_params_qlora['r']}, alpha={best_params_qlora['alpha']} (loss: {best_params_qlora['loss']:.4f})")

    # Финальное обучение QLoRa с лучшими параметрами
    model_name_final_qlora = f"QLoRa r={best_params_qlora['r']},a={best_params_qlora['alpha']} ({BASE_MODEL_NAME_UNSLOTH_QUANTIZED})"
    run_name_wandb_final_qlora = "exp3_qlora_final_training"
    wandb.init(entity='galkin-vova42', project=WANDB_PROJECT_NAME, name=run_name_wandb_final_qlora, job_type="train_qlora_final", 
               config={"experiment_type": "qlora_final", 
                       "r": best_params_qlora['r'], 
                       "alpha": best_params_qlora['alpha'], 
                       "base_model": BASE_MODEL_NAME_UNSLOTH_QUANTIZED,
                       "best_hpo_loss": best_params_qlora['loss']},
               reinit=True) # Новый run для финального обучения

    # Загружаем базовую модель заново для чистоты финального обучения
    final_base_q_model, _ = FastLanguageModel.from_pretrained(
        model_name=BASE_MODEL_NAME_UNSLOTH_QUANTIZED, max_seq_length=MAX_SEQ_LENGTH, load_in_4bit=True, token=HF_ACCESS_TOKEN
    )

    final_peft_q_model = FastLanguageModel.get_peft_model(
        final_base_q_model,
        r=best_params_qlora['r'],
        target_modules=TARGET_MODULES,
        lora_alpha=best_params_qlora['alpha'],
        lora_dropout=0,
        bias="none",
        use_gradient_checkpointing="unsloth",
        random_state=42,
    )

    trainer_q_final = SFTTrainer(
        model=final_peft_q_model,
        tokenizer=tokenizer_qlora, # Используем тот же токенизатор
        train_dataset=dataset_train_q,
        eval_dataset=dataset_test_q,
        args=SFTConfig(
            dataset_text_field="text",
            output_dir=OUTPUT_DIR_QLORA, 
            per_device_train_batch_size=2, # Как в вашем коде
            per_device_eval_batch_size=2,  # Добавим для консистентности
            gradient_accumulation_steps=4,
            warmup_steps=10, 
            max_steps=100,  # Увеличьте для полноценного обучения (например, 200-500)
            learning_rate=2e-4,
            logging_steps=1,
            optim="adamw_8bit",
            weight_decay=0.01,
            lr_scheduler_type="linear",
            seed=42,
            report_to="wandb",
            do_eval=True,
            eval_strategy="steps",
            eval_steps=20, 
            save_strategy="steps", 
            save_steps=20,
            save_total_limit=2,
            load_best_model_at_end=True, 
            metric_for_best_model="eval_loss",
            greater_is_better=False,
            fp16=not torch.cuda.is_bf16_supported(), # Для QLoRA часто bf16=True если доступно
            bf16=torch.cuda.is_bf16_supported(),
        ),
    )
    
    start_gpu_memory_qlora = round(torch.cuda.max_memory_reserved() / 1024**3, 3) if torch.cuda.is_available() else 0
    trainer_stats_qlora = trainer_q_final.train()
    
    peak_vram_train_qlora_gb = 0
    if torch.cuda.is_available():
        used_memory_qlora_train = round(torch.cuda.max_memory_reserved() / 1024**3, 3)
        peak_vram_train_qlora_gb = round(used_memory_qlora_train - start_gpu_memory_qlora, 3)
        print(f"Пиковая VRAM для обучения (QLoRa final): {peak_vram_train_qlora_gb:.3f} GB")
        wandb.log({"peak_vram_train_gb": peak_vram_train_qlora_gb})

    best_qlora_model_path = os.path.join(OUTPUT_DIR_QLORA, "best_qlora_adapters")
    final_peft_q_model.save_pretrained(best_qlora_model_path) 
    tokenizer_qlora.save_pretrained(best_qlora_model_path)
    print(f"Лучшие QLoRa адаптеры сохранены в: {best_qlora_model_path}")
    if os.path.exists(best_qlora_model_path):
        wandb.save(os.path.join(best_qlora_model_path, "*"))


    # --- Начало блока пакетной оценки для финальной QLoRa модели ---
    print(f"Оценка лучшей QLoRa модели: {model_name_final_qlora} (пакетная обработка)...")
    eval_batch_size_qlora = 8 # Как в вашем обновленном Exp1

    predictions_q_final = []
    references_q_final = []
    latencies_q_final = []
    num_samples_for_eval = min(100, len(dataset_test_raw))

    eval_data_q_final = []
    for i in range(num_samples_for_eval):
        example = dataset_test_raw[i] 
        eval_data_q_final.append({
            "user_content": example["text_with_errors"],
            "reference_text": example["corrected_text"]
        })

    if torch.cuda.is_available():
        torch.cuda.reset_peak_memory_stats()
        _ = torch.cuda.max_memory_allocated()

    device = "cuda" if torch.cuda.is_available() else "cpu"
    # final_peft_q_model уже лучшая модель и находится на device

    # Убедимся, что у токенизатора есть pad_token
    if tokenizer_qlora.pad_token is None:
        tokenizer_qlora.pad_token = tokenizer_qlora.eos_token
        # Модель уже должна быть правильно сконфигурирована Unsloth при загрузке с load_in_4bit
        # if hasattr(final_peft_q_model.config, 'pad_token_id') and final_peft_q_model.config.pad_token_id is None:
        # final_peft_q_model.config.pad_token_id = tokenizer_qlora.eos_token_id
    
    for batch_data in tqdm(create_batches(eval_data_q_final, eval_batch_size_qlora), desc=f"Оценка {model_name_final_qlora}"):
        batch_prompts_text = []
        current_references = []

        for item in batch_data:
            messages = [
                {"role": "system", "content": SYSTEM_PROMPT},
                {"role": "user", "content": item["user_content"]}
            ]
            prompt_text = tokenizer_qlora.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
            batch_prompts_text.append(prompt_text)
            current_references.append(item["reference_text"].strip())
        
        current_max_len_qlora = MAX_SEQ_LENGTH
        if hasattr(tokenizer_qlora, 'model_max_length') and tokenizer_qlora.model_max_length:
            current_max_len_qlora = min(tokenizer_qlora.model_max_length, MAX_SEQ_LENGTH)

        inputs = tokenizer_qlora(
            batch_prompts_text, 
            return_tensors="pt", 
            padding=True,
            truncation=True,
            max_length=current_max_len_qlora
        ).to(device)

        start_time = time.time()
        with torch.no_grad():
            outputs = final_peft_q_model.generate(
                **inputs, 
                max_new_tokens=min(512, MAX_SEQ_LENGTH),
                temperature=0.01, # Как в вашем Exp1
                do_sample=False,
                pad_token_id=tokenizer_qlora.pad_token_id
            )
        end_time = time.time()
        latencies_q_final.append(end_time - start_time)
        
        input_ids_lengths = inputs.attention_mask.sum(dim=1).tolist()

        batch_output_texts = []
        for i in range(outputs.shape[0]):
            generated_tokens_only = outputs[i, input_ids_lengths[i]:]
            output_text = tokenizer_qlora.decode(generated_tokens_only, skip_special_tokens=True)
            batch_output_texts.append(output_text.strip())
            
        predictions_q_final.extend(batch_output_texts)
        references_q_final.extend(current_references)
    
    peak_inference_vram_qlora_gb = 0
    if torch.cuda.is_available():
        peak_inference_vram_qlora_bytes = torch.cuda.max_memory_allocated() 
        peak_inference_vram_qlora_gb = peak_inference_vram_qlora_bytes / (1024**3)

    bleu_score_q = calculate_bleu(predictions_q_final, references_q_final)
    cer_score_q = calculate_cer(predictions_q_final, references_q_final)
    
    total_inference_time_q = sum(latencies_q_final)
    avg_latency_per_sample_q = total_inference_time_q / num_samples_for_eval if num_samples_for_eval > 0 else -1

    print(f"BLEU ({model_name_final_qlora}): {bleu_score_q:.4f}")
    print(f"CER ({model_name_final_qlora}): {cer_score_q:.4f}")
    print(f"Avg Inference Latency per sample ({model_name_final_qlora}): {avg_latency_per_sample_q:.4f} s")
    print(f"Peak VRAM during batched inference ({model_name_final_qlora}): {peak_inference_vram_qlora_gb:.3f} GB")
    # --- Конец блока пакетной оценки ---

    trainable_params_q, total_params_q = get_model_trainable_parameters(final_peft_q_model)
    
    results_q = {
        "experiment_group": "QLoRa",
        "model_name": model_name_final_qlora,
        "val_loss": trainer_q_final.state.best_model_checkpoint_loss if hasattr(trainer_q_final.state, 'best_model_checkpoint_loss') and trainer_q_final.state.best_model_checkpoint_loss is not None else -1,
        "bleu": bleu_score_q,
        "cer": cer_score_q,
        "trainable_params": trainable_params_q,
        "total_params": total_params_q, 
        "vram_train_peak_gb": peak_vram_train_qlora_gb,
        "vram_inference_peak_gb": peak_inference_vram_qlora_gb,
        "avg_inference_latency_s": avg_latency_per_sample_q,
        "model_path": best_qlora_model_path,
        "best_hpo_r": best_params_qlora['r'],
        "best_hpo_alpha": best_params_qlora['alpha'],
    }
    results_accumulator_list.append(results_q)
    wandb.log({
        "eval_bleu": bleu_score_q, "eval_cer": cer_score_q, "avg_inference_latency_per_sample": avg_latency_per_sample_q,
        "trainable_params": trainable_params_q, "total_params_base_quant": total_params_q, 
        "peak_inference_vram_gb": peak_inference_vram_qlora_gb,
        "best_val_loss_final_train": results_q["val_loss"]
    })
    
    wandb.finish()
    clear_gpu_memory(final_peft_q_model, trainer_q_final, final_base_q_model, tokenizer_qlora)

In [7]:
run_experiment_3_qlora(all_experiment_results)


--- Запуск Эксперимента 3: QLoRa ---
==((====))==  Unsloth 2025.5.6: Fast Qwen3 patching. Transformers: 4.51.3.
   \\   /|    NVIDIA GeForce RTX 4090. Num GPUs = 1. Max memory: 23.526 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.7.0+cu126. CUDA: 8.9. CUDA Toolkit: 12.6. Triton: 3.3.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.30. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


wandb: WARNING Using a boolean value for 'reinit' is deprecated. Use 'return_previous' or 'finish_previous' instead.


Подбор гиперпараметров r и alpha для QLoRa...

Тестирование QLoRa с r=16, alpha=32


Unsloth 2025.5.6 patched 28 layers with 28 QKV layers, 28 O layers and 28 MLP layers.
Unsloth: Tokenizing ["text"] (num_proc=2): 100%|██████████| 139/139 [00:01<00:00, 135.60 examples/s]
==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 555 | Num Epochs = 1 | Total steps = 30
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 10,092,544/6,000,000,000 (0.17% trained)
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


Unsloth: Will smartly offload gradients to save VRAM!


...

Лучшие QLoRa адаптеры сохранены в: ./results_qlora/best_qlora_adapters
Оценка лучшей QLoRa модели: QLoRa r=32,a=64 (unsloth/Qwen3-0.6B-unsloth-bnb-4bit) (пакетная обработка)...


Оценка QLoRa r=32,a=64 (unsloth/Qwen3-0.6B-unsloth-bnb-4bit): 0it [00:00, ?it/s]A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.
/opt/conda/lib/python3.10/site-packages/unsloth/kernels/utils.py:438: UserWarning: An output with one or more elements was resized since it had shape [1, 8, 1024], which does not match the required output shape [8, 1, 1024]. This behavior is deprecated, and in a future PyTorch release outputs will not be resized unless they have zero elements. You can explicitly reuse an out tensor t by resizing it, inplace, to zero elements with t.resize_(0). (Triggered internally at /pytorch/aten/src/ATen/native/Resize.cpp:30.)
  out = torch_matmul(X, W.t(), out = out)
/opt/conda/lib/python3.10/site-packages/unsloth/kernels/utils.py:443: UserWarning: An output with one or more elements was resized since it had shape [1, 8, 1024], which does not match t

BLEU (QLoRa r=32,a=64 (unsloth/Qwen3-0.6B-unsloth-bnb-4bit)): 0.1633
CER (QLoRa r=32,a=64 (unsloth/Qwen3-0.6B-unsloth-bnb-4bit)): 0.9105
Avg Inference Latency per sample (QLoRa r=32,a=64 (unsloth/Qwen3-0.6B-unsloth-bnb-4bit)): 3.5902 s
Peak VRAM during batched inference (QLoRa r=32,a=64 (unsloth/Qwen3-0.6B-unsloth-bnb-4bit)): 2.898 GB


avg_inference_latency_per_sample,▁
best_val_loss_final_train,▁
eval/loss,█▄▂▁▁
eval/runtime,▄▁▂█▃
eval/samples_per_second,▅█▇▁▆
eval/steps_per_second,▅█▇▁▆
eval_bleu,▁
eval_cer,▁
peak_inference_vram_gb,▁
peak_vram_train_gb,▁
total_params_base_quant,▁


Could not move an object to CPU: `.to` is not supported for `8-bit` bitsandbytes models. Please use the model as it is, since the model has already been set to the correct devices and casted to the correct `dtype`.
GPU memory cleared.


### Эксперемент 3. Full SFT + PTQ

In [12]:
model_sft, tokenizer_sft = FastLanguageModel.from_pretrained(
    model_name = BASE_MODEL_NAME_FULL_PRECISION,
    max_seq_length = 2048,   # Context length - can be longer, but uses more memory
    load_in_4bit = False,     # 4bit uses much less memory
    load_in_8bit = False,    # A bit more accurate, uses 2x memory
    full_finetuning = False, # We have full finetuning now!
)

for param in model_sft.parameters():
    param.requires_grad = True

dataset_train_sft = Dataset.from_dict(formatting_prompts_func(dataset_train_formatted, tokenizer_sft))
dataset_test_sft = Dataset.from_dict(formatting_prompts_func(dataset_test_formatted, tokenizer_sft))

OUTPUT_DIR_FULL_SFT = 'text-correction/all_exps/results_full_sft'

sft_config = SFTConfig(
    dataset_text_field="text",
    output_dir=OUTPUT_DIR_FULL_SFT,
    per_device_train_batch_size=3, # Очень маленький батч из-за VRAM
    per_device_eval_batch_size=3,
    gradient_accumulation_steps=16, # Увеличиваем GA для компенсации малого батча
    warmup_steps=10,
    max_steps=100,
    learning_rate=5e-6, # Обычно ниже для Full SFT (например, 2e-5, 1e-5, 5e-6)
    logging_steps=1,
    optim="adamw_torch",
    weight_decay=0.01,
    # num_train_epochs=2,
    lr_scheduler_type="linear",
    seed=42,
    report_to="wandb",
    fp16=False, # bf16 предпочтительнее, если доступно и модель загружена в bf16
    bf16=torch.cuda.is_bf16_supported(),
    do_eval=True,
    eval_strategy="steps",
    eval_steps=10,
    save_strategy="steps",
    save_steps=10,
    save_total_limit=2,
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
    greater_is_better=False,
    # gradient_checkpointing=True, # Можно включить здесь или Unsloth уже мог включить
    # Для очень больших моделей и ограниченной VRAM, рассмотрите DeepSpeed:
    # deepspeed="path/to/deepspeed_config.json", # если у вас есть конфиг DeepSpeed (Stage 2 или 3)
)

trainer_sft = SFTTrainer(
    model=model_sft,
    tokenizer=tokenizer_sft,
    train_dataset=dataset_train_sft,
    eval_dataset=dataset_test_sft,
    args=sft_config,
)

==((====))==  Unsloth 2025.5.6: Fast Qwen3 patching. Transformers: 4.51.3.
   \\   /|    NVIDIA GeForce RTX 4090. Num GPUs = 1. Max memory: 23.526 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.7.0+cu126. CUDA: 8.9. CUDA Toolkit: 12.6. Triton: 3.3.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.30. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Unsloth: Tokenizing ["text"] (num_proc=2): 100%|██████████| 139/139 [00:01<00:00, 116.76 examples/s]


In [13]:
trainer_sft.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 555 | Num Epochs = 10 | Total steps = 100
O^O/ \_/ \    Batch size per device = 3 | Gradient accumulation steps = 16
\        /    Data Parallel GPUs = 1 | Total batch size (3 x 16 x 1) = 48
 "-____-"     Trainable parameters = 596,049,920/596,049,920 (100.00% trained)


Step,Training Loss,Validation Loss
10,1.623700,1.649266
20,1.516800,1.494032
30,1.375800,1.389468
40,1.350300,1.312293
50,1.287000,1.268899
60,1.247700,1.245877
70,1.223900,1.234134
80,1.225700,1.229404
90,1.177200,1.228148
100,1.215100,1.227621


There were missing keys in the checkpoint model loaded: ['lm_head.weight'].


TrainOutput(global_step=100, training_loss=1.3467343986034392, metrics={'train_runtime': 474.3145, 'train_samples_per_second': 10.12, 'train_steps_per_second': 0.211, 'total_flos': 1.533264131063808e+16, 'train_loss': 1.3467343986034392})

In [26]:
eval_batch_size_qlora = 8

predictions_q_final = []
references_q_final = []
latencies_q_final = []
num_samples_for_eval = min(100, len(dataset_test_raw))

eval_data_q_final = []
for i in range(num_samples_for_eval):
    example = dataset_test_raw[i] 
    eval_data_q_final.append({
        "user_content": example["text_with_errors"],
        "reference_text": example["corrected_text"]
    })

if torch.cuda.is_available():
    torch.cuda.reset_peak_memory_stats()
    _ = torch.cuda.max_memory_allocated()

device = "cuda" if torch.cuda.is_available() else "cpu"
if tokenizer_sft.pad_token is None:
    tokenizer_sft.pad_token = tokenizer_sft.eos_token

for batch_data in tqdm(create_batches(eval_data_q_final, eval_batch_size_qlora), desc=f"Оценка sft model"):
    batch_prompts_text = []
    current_references = []

    for item in batch_data:
        messages = [
            {"role": "system", "content": SYSTEM_PROMPT},
            {"role": "user", "content": item["user_content"]}
        ]
        prompt_text = tokenizer_sft.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
        batch_prompts_text.append(prompt_text)
        current_references.append(item["reference_text"].strip())

    current_max_len_qlora = MAX_SEQ_LENGTH
    if hasattr(tokenizer_sft, 'model_max_length') and tokenizer_sft.model_max_length:
        current_max_len_qlora = min(tokenizer_sft.model_max_length, MAX_SEQ_LENGTH)

    inputs = tokenizer_sft(
        batch_prompts_text, 
        return_tensors="pt", 
        padding=True,
        truncation=True,
        max_length=current_max_len_qlora
    ).to(device)

    start_time = time.time()
    with torch.no_grad():
        outputs = model_sft.generate(
            **inputs, 
            max_new_tokens=min(512, MAX_SEQ_LENGTH),
            temperature=0.01, # Как в вашем Exp1
            do_sample=False,
            pad_token_id=tokenizer_sft.pad_token_id
        )
    end_time = time.time()
    latencies_q_final.append(end_time - start_time)

    input_ids_lengths = inputs.attention_mask.sum(dim=1).tolist()

    batch_output_texts = []
    for i in range(outputs.shape[0]):
        generated_tokens_only = outputs[i, input_ids_lengths[i]:]
        output_text = tokenizer_sft.decode(generated_tokens_only, skip_special_tokens=True)
        batch_output_texts.append(output_text.strip())

    predictions_q_final.extend(batch_output_texts)
    references_q_final.extend(current_references)

peak_inference_vram_qlora_gb = 0
if torch.cuda.is_available():
    peak_inference_vram_qlora_bytes = torch.cuda.max_memory_allocated() 
    peak_inference_vram_qlora_gb = peak_inference_vram_qlora_bytes / (1024**3)

bleu_score_q = calculate_bleu(predictions_q_final, references_q_final)
cer_score_q = calculate_cer(predictions_q_final, references_q_final)

total_inference_time_q = sum(latencies_q_final)
avg_latency_per_sample_q = total_inference_time_q / num_samples_for_eval if num_samples_for_eval > 0 else -1

print(f"BLEU {bleu_score_q:.4f}")
print(f"CER {cer_score_q:.4f}")

Оценка sft model: 0it [00:00, ?it/s]A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.
/opt/conda/lib/python3.10/site-packages/unsloth/kernels/utils.py:438: UserWarning: An output with one or more elements was resized since it had shape [1, 8, 1024], which does not match the required output shape [8, 1, 1024]. This behavior is deprecated, and in a future PyTorch release outputs will not be resized unless they have zero elements. You can explicitly reuse an out tensor t by resizing it, inplace, to zero elements with t.resize_(0). (Triggered internally at /pytorch/aten/src/ATen/native/Resize.cpp:30.)
  out = torch_matmul(X, W.t(), out = out)
Оценка sft model: 1it [00:15, 15.25s/it]A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.
Оценка sft model:

BLEU 0.2139
CER 0.7020


In [14]:
from transformers import GPTQConfig, AutoTokenizer

In [18]:
from auto_gptq import AutoGPTQForCausalLM, BaseQuantizeConfig
import logging

pretrained_model_dir = "results_full_sft/checkpoint-100"
quantized_model_dir = "quantized"

quantize_config = BaseQuantizeConfig(
    bits=4,
    group_size=128,
    desc_act=False,
)

try:
    model = AutoGPTQForCausalLM.from_pretrained(pretrained_model_dir, quantize_config)
except Exception as e:
    print(e)

qwen3 isn't supported yet.


In [59]:
batch = dataset_test_sft['text'][2:3]

inputs = tokenizer_sft(
    batch, 
    return_tensors="pt", 
    padding=True,
    truncation=True,
    max_length=current_max_len_qlora
).to(device)

with torch.no_grad():
    outputs = model_sft.generate(
        **inputs, 
        max_new_tokens=min(512, MAX_SEQ_LENGTH),
        do_sample=False,  # Greedy decoding
        pad_token_id=tokenizer_sft.pad_token_id,
        eos_token_id=tokenizer_sft.eos_token_id  # Often helpful to add
    )

generated_text = tokenizer_sft.decode(outputs[0], skip_special_tokens=True)
print(generated_text)

system
Отредактируйте следующий текст. Очень важно, чтобы вы сохранили как можно больше оригинального текста. Вы можете вносить небольшие и нечастые правки, которые исправляют только грамматические, орфографические или пунктационные ошибки. Если в предложении нет ошибок, повторите предложение дословно. Не объясняйте свои правки и не добавляйте примечания. Используйте только русский язык.
user
Генеральный диретор симферопольской «Таврии» Александер Гайдаш в перерыве матча против «Гвардейца» (Гвардейское Симферопольский район) напал на одного из помошников главного арбитра встречи Руслана Селимова. Об этом сообщает Matcday. Команды встречались в игре третьего тура Всекрымского турнира-2015. Гайдаш ворвался в судейскую и нанес Селимову несколько ударов. Почти сразу был вызван наряд полиции, но главный арбитр встречи Дмитрий Грачев принял решение, не останавливать поединок. Вышел на матч, и сам пострадавший помощник рефери. После игры, был составлен протокол. Свидетелями выступили арбитр, 

In [71]:
from difflib import ndiff
from IPython.core.display import display, HTML

def format_diffs(original, corrected):
    diffs = list(ndiff(original.split(), corrected.split()))
    formatted_text = ""
    for diff in diffs:
        if diff.startswith('-'):
            formatted_text += f'<span style="color: red; text-decoration: line-through;">{diff[2:]}</span> '
        elif diff.startswith('+'):
            formatted_text += f'<span style="color: green;">{diff[2:]}</span> '
        elif diff.startswith(' '):
            formatted_text += diff[2:] + ' '
    return formatted_text

input_text = 'Генеральный диретор симферопольской «Таврии» Александер Гайдаш в перерыве матча против «Гвардейца» (Гвардейское Симферопольский район) напал на одного из помошников главного арбитра встречи Руслана Селимова. Об этом сообщает Matcday. Команды встречались в игре третьего тура Всекрымского турнира-2015. Гайдаш ворвался в судейскую и нанес Селимову несколько ударов. Почти сразу был вызван наряд полиции, но главный арбитр встречи Дмитрий Грачев принял решение, не останавливать поединок. Вышел на матч, и сам пострадавший помощник рефери. После игры, был составлен протокол. Свидетелями выступили арбитр, и второй помощник. Инспектор матча Владимир Куцев, отметил эпизод в рапорте и отправил его в Федерацию футбола Крыма. Теперь Гайдашу грозят серьёзные санкции. Агрессивная реакция Александра Гайдаша, на действия помощника рефери была спровоцирована решением ассистента об отмене, в первом тайме гола «Таврии» за нападение на вратаря соперников. Встреча завершилась победой «Гвардейца» со счетом 1:0. В августе 2011 года в перерыве матча чемпионата России по футболу между махачкалинским «Анжи» и московским «Динамо» (2:1), неизвестный сломал дверь в коридоре ведущем в судейскую комнату и комнату делегата матча. Подозрения пали на главного тренера команды из Дагестана Гаджи Гаджиева но никаких доказательств так и не было представлено.'
output_text = 'Генеральный директор симферопольской «Таврии» Александр Гайдаш в перерыве матча против «Гвардейца» (Гвардейское, Симферопольский район) напал на одного из помощников главного арбитра встречи Руслана Селимова. Об этом сообщает Matcday. Команды встречались в игре третьего тура Всекрымского турнира-2015. Гайдаш ворвался в судейскую и нанес Селимову несколько ударов. Почти сразу был вызван наряд полиции, но главный арбитр встречи Дмитрий Грачев принял решение не останавливать поединок. Вышел на матч и сам пострадавший помощник рефери. После игры был составлен протокол. Свидетелями выступили арбитр и второй помощник. Инспектор матча Владимир Куцев отметил эпизод в рапорте и отправил его в Федерацию футбола Крыма. Теперь Гайдашу грозят серьезные санкции. Агрессивная реакция Александра Гайдаша на действия помощника рефери была спровоцирована решением ассистента об отмене в первом тайме гола «Таврии» за нападение на вратаря соперников. Встреча завершилась победой «Гвардейца» со счетом 1:0. В августе 2011 года в перерыве матча чемпионата России по футболу между махачкалинским «Анжи» и московским «Динамо» (2:1) неизвестный сломал дверь в коридоре, ведущем в судейскую комнату и комнату делегата матча. Подозрения пали на главного тренера команды из Дагестана Гаджи Гаджиева, но никаких доказательств так и не было представлено.'

display(HTML(format_diffs(input_text, output_text)))

/tmp/ipykernel_1274/1980763572.py:2: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython.display
  from IPython.core.display import display, HTML


In [77]:
model, tokenizer = FastLanguageModel.from_pretrained(
        model_name='results_qlora/checkpoint-100',
        max_seq_length=MAX_SEQ_LENGTH,
        dtype=None, # Автоматический выбор или torch.float16, torch.bfloat16
        load_in_4bit=False,
        load_in_8bit=False,
        token=HF_ACCESS_TOKEN,
    )

==((====))==  Unsloth 2025.5.6: Fast Qwen3 patching. Transformers: 4.51.3.
   \\   /|    NVIDIA GeForce RTX 4090. Num GPUs = 1. Max memory: 23.526 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.7.0+cu126. CUDA: 8.9. CUDA Toolkit: 12.6. Triton: 3.3.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.30. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Unsloth 2025.5.6 patched 28 layers with 28 QKV layers, 28 O layers and 28 MLP layers.


In [78]:
batch = dataset_test_sft['text'][2:3]

inputs = tokenizer(
    batch, 
    return_tensors="pt", 
    padding=True,
    truncation=True,
    max_length=current_max_len_qlora
).to(device)

with torch.no_grad():
    outputs = model.generate(
        **inputs, 
        max_new_tokens=min(512, MAX_SEQ_LENGTH),
        do_sample=False,  # Greedy decoding
        pad_token_id=tokenizer_sft.pad_token_id,
        eos_token_id=tokenizer_sft.eos_token_id  # Often helpful to add
    )

generated_text = tokenizer_sft.decode(outputs[0], skip_special_tokens=True)
print(generated_text)

system
Отредактируйте следующий текст. Очень важно, чтобы вы сохранили как можно больше оригинального текста. Вы можете вносить небольшие и нечастые правки, которые исправляют только грамматические, орфографические или пунктационные ошибки. Если в предложении нет ошибок, повторите предложение дословно. Не объясняйте свои правки и не добавляйте примечания. Используйте только русский язык.
user
Генеральный диретор симферопольской «Таврии» Александер Гайдаш в перерыве матча против «Гвардейца» (Гвардейское Симферопольский район) напал на одного из помошников главного арбитра встречи Руслана Селимова. Об этом сообщает Matcday. Команды встречались в игре третьего тура Всекрымского турнира-2015. Гайдаш ворвался в судейскую и нанес Селимову несколько ударов. Почти сразу был вызван наряд полиции, но главный арбитр встречи Дмитрий Грачев принял решение, не останавливать поединок. Вышел на матч, и сам пострадавший помощник рефери. После игры, был составлен протокол. Свидетелями выступили арбитр, 

In [80]:
from difflib import ndiff
from IPython.core.display import display, HTML

def format_diffs(original, corrected):
    diffs = list(ndiff(original.split(), corrected.split()))
    formatted_text = ""
    for diff in diffs:
        if diff.startswith('-'):
            formatted_text += f'<span style="color: red; text-decoration: line-through;">{diff[2:]}</span> '
        elif diff.startswith('+'):
            formatted_text += f'<span style="color: green;">{diff[2:]}</span> '
        elif diff.startswith(' '):
            formatted_text += diff[2:] + ' '
    return formatted_text

input_text = 'Генеральный диретор симферопольской «Таврии» Александер Гайдаш в перерыве матча против «Гвардейца» (Гвардейское Симферопольский район) напал на одного из помошников главного арбитра встречи Руслана Селимова. Об этом сообщает Matcday. Команды встречались в игре третьего тура Всекрымского турнира-2015. Гайдаш ворвался в судейскую и нанес Селимову несколько ударов. Почти сразу был вызван наряд полиции, но главный арбитр встречи Дмитрий Грачев принял решение, не останавливать поединок. Вышел на матч, и сам пострадавший помощник рефери. После игры, был составлен протокол. Свидетелями выступили арбитр, и второй помощник. Инспектор матча Владимир Куцев, отметил эпизод в рапорте и отправил его в Федерацию футбола Крыма. Теперь Гайдашу грозят серьёзные санкции. Агрессивная реакция Александра Гайдаша, на действия помощника рефери была спровоцирована решением ассистента об отмене, в первом тайме гола «Таврии» за нападение на вратаря соперников. Встреча завершилась победой «Гвардейца» со счетом 1:0. В августе 2011 года в перерыве матча чемпионата России по футболу между махачкалинским «Анжи» и московским «Динамо» (2:1), неизвестный сломал дверь в коридоре ведущем в судейскую комнату и комнату делегата матча. Подозрения пали на главного тренера команды из Дагестана Гаджи Гаджиева но никаких доказательств так и не было представлено.'
output_text = 'Генеральный директор симферопольской «Таврии» Александр Гайдаш в перерыве матча против «Гвардейца» (Гвардейское, Симферопольский район) напал на одного из помощников главного арбитра встречи Руслана Селимова. Об этом сообщает Matcday. Команды встречались в игре третьего тура Всекрымского турнира-2015. Гайдаш ворвался в судейскую и нанес Селимову несколько ударов. Почти сразу был вызван наряд полиции, но главный арбитр встречи Дмитрий Грачев принял решение не останавливать поединок. Вышел на матч и сам пострадавший помощник рефери. После игры был составлен протокол. Свидетелями выступили арбитр и второй помощник. Инспектор матча Владимир Куцев отметил эпизод в рапорте и отправил его в Федерацию футбола Крыма. Теперь Гайдашу грозят серьезные санкции. Агрессивная реакция Александра Гайдаша на действия помощника рефери была спровоцирована решением ассистента об отмене в первом тайме гола «Таврии» за нападение на вратаря соперников. Встреча завершилась победой «Гвардейца» со счетом 1:0. В августе 2011 года в перерыве матча чемпионата России по футболу между махачкалинским «Анжи» и московским «Динамо» (2:1) неизвестный сломал дверь в коридоре, ведущем в судейскую комнату и комнату делегата матча. Подозрения пали на главного тренера команды из Дагестана Гаджи Гаджиева, но никаких доказательств так и не было представлено.  #Турнир #Футбол #Симферополь #Гвардейцы #Гайда'

display(HTML(format_diffs(input_text, output_text)))

/tmp/ipykernel_1274/2496646625.py:2: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython.display
  from IPython.core.display import display, HTML
